In [79]:
import os
import json
import ast
import math
import pandas as pd
import gzip
import pickle as pkl
from collections import defaultdict
from datetime import datetime

In [80]:
os.getcwd()

'e:\\Workspaces\\SASRec.pytorch\\notebooks'

In [81]:
ROOT_DIR = os.path.join(os.getcwd(), "..", "data", "Steam")

In [82]:
ROOT_DIR

'e:\\Workspaces\\SASRec.pytorch\\notebooks\\..\\data\\Steam'

In [83]:
# def parse(g):
#     for l in g:
#         yield eval(l)


# countU = defaultdict(lambda: 0)
# countP = defaultdict(lambda: 0)
# line = 0

# dataset_name = "Steam"
# json_file_path = os.path.join(ROOT_DIR, f"reviews_{dataset_name}.json.gz")

# with gzip.open(json_file_path, "r") as g:
#     line = 0
#     for l in parse(g):
#         line += 1
#         if line % 100000 == 0:
#             print(line)

#         product_id = int(l["product_id"])
#         username = l["username"]
#         time = datetime.strptime(l["date"], "%Y-%m-%d")
#         countU[username] += 1
#         countP[product_id] += 1

In [84]:
# keys = set(countP.keys())
# ids = set(df['id'].unique())

In [85]:
# df[df['id']==340]

In [86]:
# keys - ids

In [87]:
with open(os.path.join(ROOT_DIR, "steam_games.json")) as json_file:
    lines = json_file.read().splitlines()
records = [ast.literal_eval(line) for line in lines]

raw_df = pd.DataFrame(records)

In [88]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   discount_price  225 non-null    float64
 8   reviews_url     32133 non-null  object 
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  bool   
 12  id              32133 non-null  object 
 13  developer       28836 non-null  object 
 14  sentiment       24953 non-null  object 
 15  metascore       2677 non-null   object 
dtypes: bool(1), float64(1), object(14)
memory usage: 3.7+ MB


In [92]:
df = raw_df[['id', 'sentiment', 'genres', 'specs', 'price', 'early_access']]

In [93]:
#fix id
df.dropna(subset = ['id'], inplace=True)
df['id'] = df['id'].astype(int)

itemmap = {}
with open(os.path.join(ROOT_DIR, "itemmap.pkl"), 'rb') as f:
    itemmap = pkl.load(f)

df['id'] = df['id'].apply(lambda x: itemmap[x] if x in  itemmap else None)
df.dropna(subset = ['id'], inplace=True)
df = df.drop_duplicates(subset=['id'])

df['id'] = df['id'].astype(int)
df = df.reset_index()
print(len(df))

13047
<ipython-input-93-9604afbdf72d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset = ['id'], inplace=True)
<ipython-input-93-9604afbdf72d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = df['id'].astype(int)
<ipython-input-93-9604afbdf72d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[

In [94]:
df

,index,id,sentiment,genres,specs,price,early_access
0,1,3061,Mostly Positive,"[Free to Play, Indie, RPG, Strategy]","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False
1,2,2019,Mostly Positive,"[Casual, Free to Play, Indie, Simulation, Sports]","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False
2,5,10834,Mixed,"[Action, Adventure, Simulation]","[Single-player, Steam Achievements]",3.99,False
3,27,6827,Very Positive,"[Action, Indie, Racing]","[Single-player, Multi-player, Steam Trading Ca...",9.99,False
4,28,3583,Overwhelmingly Positive,[Action],"[Single-player, Multi-player, Valve Anti-Cheat...",9.99,False
...,...,...,...,...,...,...,...
13042,32116,6392,Overwhelmingly Positive,[Action],"[Single-player, Multi-player, Steam Trading Ca...",14.99,False
13043,32117,2950,Very Positive,[Action],"[Single-player, Multi-player, Valve Anti-Cheat...",9.99,False
13044,32124,10483,4 user reviews,"[Adventure, Indie]","[Single-player, Steam Achievements]",0.99,False
13045,32129,9334,2 user reviews,"[Action, Adventure, Casual, Indie]","[Single-player, Steam Achievements, Steam Cloud]",1.99,False


In [95]:
#fix sentiment
sentiments = ['Unknown', 'Overwhelmingly Negative', "Very Negative", "Negative", "Mostly Negative", "Mixed", "Mostly Positive", "Positive" , "Very Positive" ,"Overwhelmingly Positive"]
sentiments_dict = {
    'Unknown': -1,
    'Overwhelmingly Negative': 0,
    'Very Negative': 1,
    'Negative': 2,
    'Mostly Negative': 3,
    'Mixed': 4,
    'Mostly Positive': 5,
    'Positive': 6,
    'Very Positive': 7,
    'Overwhelmingly Positive': 8,
}
df['sentiment'] = df['sentiment'].fillna('Unknown')
df['sentiment'] = df['sentiment'].apply(lambda x: x if x in sentiments else 'Unknown')
df['sentiment'] = df['sentiment'].apply(lambda x: sentiments_dict[x])
df['sentiment'] = df['sentiment'].astype(int)

In [96]:
#fix genres
genres = set()
for item in list(raw_df["genres"].dropna()):        
    genres = genres | set(item)
print(len(genres))

def is_in_genere(genre, x):
    try:
        return 1 if genre in x else 0
    except:
        return 0

for genre in genres:
    df[f'genre_{genre}'] = df['genres'].apply(lambda x: is_in_genere(genre=genre, x=x))

df.drop(columns=['genres'],inplace=True)

22


In [97]:
#fix specs
specs = set()
for item in list(raw_df["specs"].dropna()):        
    specs = specs | set(item)
print(len(specs))

def is_in_spec(spec, x):
    try:
        return 1 if spec in x else 0
    except:
        return 0

for spec in specs:
    df[f'spec_{spec}'] = df['specs'].apply(lambda x: is_in_spec(spec, x))

df.drop(columns=['specs'],inplace=True)

40


In [98]:
df

,index,id,sentiment,price,early_access,genre_Sports,genre_Simulation,genre_Early Access,genre_Video Production,genre_Software Training,...,spec_Valve Anti-Cheat enabled,spec_Captions available,spec_Steam Achievements,spec_Steam Cloud,spec_Cross-Platform Multiplayer,spec_Online Co-op,spec_Mods (require HL1),spec_Multi-player,spec_Shared/Split Screen,spec_Steam Turn Notifications
0,1,3061,5,Free To Play,False,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
1,2,2019,5,Free to Play,False,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,5,10834,4,3.99,False,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,27,6827,7,9.99,False,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,3583,8,9.99,False,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13042,32116,6392,8,14.99,False,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
13043,32117,2950,7,9.99,False,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
13044,32124,10483,-1,0.99,False,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
13045,32129,9334,-1,1.99,False,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0


In [99]:
#fix price
# strange_prices = ['Free to Play', 'nan','Free','Free Demo', 'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo', 'Free Mod', 'Install Theme', 'Third-party', 'Play Now', 'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Starting at $499.00', 'Starting at $449.00', 'Free to Try', 'Free Movie', 'Free to Use']
free_prices = ['Free to Play', 'Free', 'Free Demo', 'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo', 'Free Mod', 'Install Theme', 'Play Now', 'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Free to Try', 'Free Movie', 'Free to Use', 'Free To Play	', "'Free To Play'", 'Free To Play']
free_prices = free_prices + [df.iloc[1]['price']]
df['price'] = df['price'].apply(lambda x: 0 if x in free_prices else x)
df['price'] = df['price'].apply(lambda x: 0 if x == 'Third-party' else x)
df['price'] = df['price'].apply(lambda x: 499.0 if x == 'Starting at $499.00' else x)
df['price'] = df['price'].apply(lambda x: 449.0 if x == 'Starting at $449.00' else x)
df['price'] = df['price'].fillna(-1)
# df['price'].iloc[0] = 0
# df.dropna(subset = ['price'], inplace=True)

# df['price']=df['price'].apply(lambda x:round(x,2))

In [100]:
df[df['price'] == "Free To Play"]

,index,id,sentiment,price,early_access,genre_Sports,genre_Simulation,genre_Early Access,genre_Video Production,genre_Software Training,...,spec_Valve Anti-Cheat enabled,spec_Captions available,spec_Steam Achievements,spec_Steam Cloud,spec_Cross-Platform Multiplayer,spec_Online Co-op,spec_Mods (require HL1),spec_Multi-player,spec_Shared/Split Screen,spec_Steam Turn Notifications


In [101]:
# fix early_access
df['early_access'] = df['early_access'].apply(lambda x: 1 if x else 0)

In [102]:
df[df.isnull().any(axis=1)]

,index,id,sentiment,price,early_access,genre_Sports,genre_Simulation,genre_Early Access,genre_Video Production,genre_Software Training,...,spec_Valve Anti-Cheat enabled,spec_Captions available,spec_Steam Achievements,spec_Steam Cloud,spec_Cross-Platform Multiplayer,spec_Online Co-op,spec_Mods (require HL1),spec_Multi-player,spec_Shared/Split Screen,spec_Steam Turn Notifications


In [103]:
df.to_csv(os.path.join(ROOT_DIR, "items_info.csv"), index=False)

In [104]:
df

,index,id,sentiment,price,early_access,genre_Sports,genre_Simulation,genre_Early Access,genre_Video Production,genre_Software Training,...,spec_Valve Anti-Cheat enabled,spec_Captions available,spec_Steam Achievements,spec_Steam Cloud,spec_Cross-Platform Multiplayer,spec_Online Co-op,spec_Mods (require HL1),spec_Multi-player,spec_Shared/Split Screen,spec_Steam Turn Notifications
0,1,3061,5,0.00,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
1,2,2019,5,0.00,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,5,10834,4,3.99,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,27,6827,7,9.99,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,3583,8,9.99,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13042,32116,6392,8,14.99,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
13043,32117,2950,7,9.99,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
13044,32124,10483,-1,0.99,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
13045,32129,9334,-1,1.99,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0


In [105]:
from sklearn.decomposition import PCA

pca_df = df.drop(columns=['id'])
pca = PCA(n_components=10)
pca_np = pca.fit_transform(pca_df)
pca_df = pd.DataFrame(pca_np)
pca_df['id'] = df['id']
pca_df = pca_df[['id' ,0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]
pca_df.to_csv(os.path.join(ROOT_DIR, "items_info_pca_10.csv"), index=False)

In [106]:
pca_df

,id,0,1,2,3,4,5,6,7,8,9
0,3061,16432.400353,-9.526871,-0.858684,-0.007223,-0.553376,1.080186,0.947382,-0.912903,0.088379,0.429949
1,2019,16431.400354,-9.531730,-0.788061,-0.871618,-0.480422,0.774918,0.532814,0.652818,0.067127,0.657292
2,10834,16428.400375,-5.548444,0.238338,-0.288806,-0.002880,-0.570166,-0.250191,-0.259052,-0.197300,-0.190218
3,6827,16406.400382,0.490998,-2.654488,-0.206489,-0.636648,-0.265038,0.216159,0.218816,-0.288742,0.198579
4,3583,16405.400374,0.512646,-3.641782,-1.005539,-0.903518,-0.416534,-0.405695,0.061184,-0.239605,0.187734
...,...,...,...,...,...,...,...,...,...,...,...
13042,6392,-15682.599602,5.651712,-3.388086,-0.521265,-0.629375,-0.241820,-0.203292,-0.076231,-0.494414,0.471103
13043,2950,-15683.599619,0.637768,-2.447050,-0.921916,-0.857039,-0.377570,-0.381291,0.072069,-0.207651,0.200201
13044,10483,-15690.599605,-8.489870,5.386244,0.072263,0.594153,-0.287338,0.047027,-0.263163,0.198884,-0.204706
13045,9334,-15695.599600,-7.493152,5.385161,0.739705,0.452658,-0.349946,-0.049873,0.224684,-0.059074,0.040273


In [107]:
from sklearn.cluster import FeatureAgglomeration

fagg_df = df.drop(columns=['id'])
fagg = FeatureAgglomeration(n_clusters=10)
fagg_np = fagg.fit_transform(fagg_df)
fagg_df = pd.DataFrame(fagg_np)
fagg_df['id'] = df['id']
fagg_df = fagg_df[['id' ,0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]
fagg_df.to_csv(os.path.join(ROOT_DIR, "items_info_fagg_10.csv"), index=False)

In [108]:
fagg_df

,id,0,1,2,3,4,5,6,7,8,9
0,3061,0.0,1.0,0.428571,1.0,0.0,0.00,0.000000,1.0,0.086957,5.0
1,2019,0.5,1.0,0.285714,0.0,0.0,0.00,0.000000,2.0,0.108696,5.0
2,10834,0.5,0.5,0.142857,0.5,1.0,3.99,0.000000,5.0,0.000000,4.0
3,6827,0.0,1.0,0.142857,0.5,1.0,9.99,0.000000,27.0,0.021739,7.0
4,3583,0.0,0.5,0.142857,0.0,1.0,9.99,0.000000,28.0,0.021739,8.0
...,...,...,...,...,...,...,...,...,...,...,...
13042,6392,0.0,0.5,0.142857,0.5,1.0,14.99,0.000000,32116.0,0.000000,8.0
13043,2950,0.0,0.5,0.142857,0.0,1.0,9.99,0.000000,32117.0,0.021739,7.0
13044,10483,0.5,1.0,0.000000,0.5,0.0,0.99,0.000000,32124.0,0.000000,-1.0
13045,9334,1.0,1.0,0.000000,0.5,1.0,1.99,0.333333,32129.0,0.000000,-1.0
